# <mark> <b> > 1. </b> Mapeamento e analise do  Pipeline </mark>

<b>1_map_analise_pipeline_v0.ipynb </b> |  Atual notebook com as funçoes de mapeamento e analise do pipeline

### Modules & config

In [70]:
import os
import sys
import shutil
import platform
import subprocess
from io import StringIO
from pathlib import Path
from urllib import response

from outlook_msg import Message
import extract_msg
import zipfile
from pyunpack import Archive
import py7zr

import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import PyPDF2

import csv
import json
import pandas as pd

import uuid
import hashlib

import locale
import time, copy
from pytz import timezone
from datetime import datetime, timezone, timedelta

import cv2
import fitz  # Módulo PyMuPDF
from PIL import Image
from PIL import ImageFont
from PIL import Image, ImageDraw
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar
import matplotlib.pyplot as plt
from pdf2image import convert_from_path

import pytesseract

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Modulos da solucao
import modules.cronometro as cron
import modules.utils as utl
import modules.trata_pdf as tpdf


In [10]:
import spacy
from spacy import displacy
import pandas as pd
from spacy.tokens import Span
from spacy.matcher import Matcher
from spacy.tokens import Token
from spacy.language import Language

nlp = spacy.load("pt_core_news_sm")

ner = nlp.remove_pipe('ner')
ruler = nlp.add_pipe("entity_ruler")

In [229]:
#### Config - Mapeamento e Analise do pipeline

# 1. XXX Path para planilha de processamento de batches
conf_export_plan_path = 'processamentos/processamento_batches/df_conf_export_batch.xlsx'

# 2. XXX  Tratando nome de carga do df_processamento
map_analise_path = "processamentos/mapeamento_analise"

# 3. XXX  prefixo de nome do arquivo de exportaçao
df_root_pipe_file = "df_root_"

# 4. XXX Tipos de documentos para extracao
tipo_documento_path = "config/tipo_documentos"

# 5. XXX Path para tipo de documento patterns
tipo_documento_patterns_path = "config/tipo_documentos/patterns"


# 5. Path para documentos para extracao
documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento"


# Paths de trabalho para Raster_PDF
raster_process_pdf_path = 'processamentos/temp/pdf'
raster_process_txt_path = 'processamentos/temp/txt'


# 6. XXX Nome do caminha para dict Tipo de documento
config_tipo_doc_path = "config/tipo_documentos/tipo_documento.json"





# 6. IMPORTANTE - MUDOU - Path para gestao de imagens resized
image_resized_path = "processamentos/temp/images/processadas"


#### Config - E-mail
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/11_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/13_attachments'


#### Config - messages
# 3. Caminho do arquivo uma mensagem especifica
msg_outros_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/12_messages'

# 4. Path para arquivos recebidos manualmente
arquivos_recebidos_path = 'pipeline_extracao_documentos/1_emails_documentos_recebidos/14_documentos_recebidos'


# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

In [41]:
def show_ent_new(text, patterns):
    #nlp = spacy.blank("pt")
    #ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    doc = nlp(text)
    
    tokens = []
    ents = []
    
    for ent in doc.ents:
        span = doc.char_span(ent.start_char, ent.end_char, label=ent.label_)
        ents.append(span)
        
    for token in doc:
        start = token.idx
        end = start + len(token)
        tokens.append((token.text, start, end))
        
    return doc, tokens, ents


# Funcoes para salvar e carregar entity ruler patterns

def write_patterns_to_file(patterns, colors, filename):
    data = {"patterns": patterns, "colors": colors}
    with open(filename, "w") as f:
        json.dump(data, f, ensure_ascii=True, indent=2)
        

        
def load_patterns_and_colors(filename):
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
        patterns = data["patterns"]
        colors = data["colors"]
    return patterns, colors 



def save_tipo_doc_to_file(dic, filename):
    with open(filename, 'w') as f:
        json.dump(dic, f) 
        
        
def load_dict_from_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)
        

### Funcoes para Analise do pipeline

In [ ]:
valor = 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e'
valor.lower()

In [159]:
tipo_documento_dict = {}

tipo_documento_dict = {
    'nfs_e': {
            'descricao': 'nota fiscal de serviços eletrônica - nfs-e',
            'matcher_pattern_path': path_matches_pattern_json,
            'matcher_pattern_dict': matcher_pattern_dict,
            'entity_ruler_pattern_path': entityruler_file_path,
            'entity_ruler_colors': colors,
            'entity_ruler_patterns': patterns,
            'sample_content': """27/07/2023, 15:12 Nota Fiscal de Serviços Eletrônica (NFSe) https://nfe.mesquita.rj.gov.br 1/1 PREFEITURA MUNICIPAL DE MESQUITA SECRETARIA MUNICIPAL DA FAZENDA NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Número da Nota: 20234 Competência: Julho/2023 Data e Hora da Emissão: 27/07/2023 15:11:00 Código Verificação: 4FDA9FBAE PRESTADOR DE SERVIÇOS CPF/CNPJ: 50.921.369/0001-05 Inscrição Municipal: 952538 Telefone: 2297268232.. Inscrição Estadual: Nome/Razão Social: MEDSORIA CLINICA DE AVALIACAO MEDICA E PSICOLOGICA DO TRAFEGO DE MESQUITA LTDA Nome de Fantasia: Endereço: RUA PROCOPIO ,631 LOJA A ,SANTO ELIAS - Mesquita-RJ E-mail: LARA_VSORIA@HOTMAIL.COM TOMADOR DE SERVIÇOS CPF/CNPJ: 06.047.087/0033-16 | INSC:MUNICIPAL: RG: Telefone: Inscrição Estadual: Nome/Razão Social: REDE D'OR SAO LUIZ S.A. Endereço: OLINDA ELLIS N° 93 BAIRRO: CAMPO GRANDE CIDADE: RIO DE JANEIRO - RJ CEP: 23045160 E-mail: Não Informado DISCRIMINAÇÃO DOS SERVIÇOS Ref a Plantões de Janeiro, 36h no Setor de Radiologia - Médica: Lara Veiga Soria Catuladeira. VALOR TOTAL DA NOTA: R$ 4.280,16 CNAE - 8630502 - ATIVIDADE MÉDICA AMBULATORIAL COM RECURSOS PARA REALIZAÇÃO DE EXAMES COMPLEMENTARES Item da Lista de Serviços - 4.03 - HOSPITAIS, CLÍNICAS, LABORATÓRIOS, SANATÓRIOS, MANICÔMIOS, CASAS DE SAÚDE, PRONTOS-SOCORROS, AMBULATÓRIOS E CONGÊNERES. VALOR SERVIÇOS: R$ 4.280,16 VALOR DEDUÇÃO: R$ 0,00 DESC. INCOND: R$ 0,00 BASE DE CÁLCULO: R$ 4.280,16 ALÍQUOTA: 2,01% VALOR ISS: R$ 86,03 VALOR ISS RETIDO: R$ 0,00 DESC. COND: R$ 0,00 ____________________________________________________________________ VALOR PIS: R$ 0,00 VALOR COFINS: R$ 0,00 VALOR IR: R$ 0,00 VALOR INSS: R$ 0,00 VALOR CSLL: R$ 0,00 OUTRAS RETENÇÕES: R$ 0,00 VALOR LÍQUIDO: R$ 4.280,16 DADOS COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS Exigivel REGIME TRIBUTAÇÃO Sociedade Limitada SIMPLES NACIONAL Sim ( 2,01% ) ISSQN RETIDO Não LOCAL. PRESTAÇÃO SERVIÇO Mesquita - RJ LOCAL INCIDÊNCIA Mesquita - RJ Observação: LEI DA TRANSPARÊNCIA FISCAL NR. 12.741, DE 8 DE DEZEMBRO DE 2012. - PRESTADOR OPTANTE DO SIMPLES NACIONAL (ALÍQUOTA: 2,01 %) ESTA NFS-E FOI EMITIDA EM SUBSTITUIÇÃO À NFS-E 20231 Valor Aproximado dos Tributos Federais R$ 575,68 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíq 0,00 IBPT) e Municipal de R$ 95,45 (Alíq IBPT 2,23 IBPT)""",
            'official_name': 'Official Document Name',
            'min_score_similarity_analise': 0.60,
            'min_score_similarity_documento': 0.7
    }
}    

In [ ]:
tipo_documento_dict

In [161]:
novo_tipo_documento = {
    'lista_nota': {
            'descricao': 'Listagem de Notas Fiscais',
            'matcher_pattern_path': path_matches_pattern_json,
            'matcher_pattern_dict': matcher_pattern_dict,
            'entity_ruler_pattern_path': entityruler_file_path,
            'entity_ruler_colors': colors,
            'entity_ruler_patterns': patterns,
            'sample_content': """02/08/23, 16:33 Nota Fiscal de Serviços Eletrônica (NFSe) Número da Nota: PREFEITURA MUNICIPAL DE MAGE ia Competência: SECRETARIA MUNICIPAL DA FAZENDA Agosto/2023 NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Data e Hora da Emissão: 02/08/2023 16:29:00 Código Verificação: OCDD6FB51 PRESTADOR DE SERVIÇOS CPF/CNPJ: Inscrição Municipal: Telefone: Inscrição Estadual: 33.462.862/0001-95 li: 4337 al Nome/Razão Social: SÃO MARCOS TERRAPLENAGEM E CONSTRUÇÃO LTDA. Nome de Fantasia: Endereço: RUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ E-mail: TOMADOR DE SERVIÇOS CPF/CNPJ: 29.115.474/0001-60 HINSC:MUNICIPAL: RG: | Telefone: Inscrição Estadual: Nome/Razão Social: MUNICÍPIO DE MACAE Endereço: ; PRESIDENTE FELICIANO SODRE Nº 534 PAÇO E-mail: NÃO INFORMADO MUNICIPAL BAIRRO: CENTRO CIDADE: MACAE ; - RJ CEP: 27913080 DISCRIMINAÇÃO DOS SERVIÇOS Serviço de Recomposição do Pavimento Asfáltico (Tapa Buraco), na cidade de Macaé/RJ, abrangendo Perímetro Urbano, Região Serrana, conf Boletim de Medição 08 — CO 022/2022 — Contrato 045/2022 - . Empenhos 501/2023 e 502/2023. M. Obra (10%)= R$ 104.468,45 Retenção p/ Prev. Social=R$ 11.491,53 Banco: ITAU Agência: 0726 C/Corrente: 35410-3 VALOR TOTAL DA NOTA: R$ 1.044.684,55 CNAE - 4313400 - OBRAS DE TERRAPLENAGEM . é o Item da Lista de Serviços - 7.02 - EXECUÇÃO, POR ADMINISTRAÇÃO, EMPREITADA OU SUBEMPREITADA, DE OBRAS DE CONSTRUÇÃO CIVIL, HIDRAULICA OU ELETRICA E DE OUTRAS OBRAS SEMELHANTES, INCLUSIVE SONDAGEM, PERFURAÇÃO DE POÇOS, ESCAVAÇÃO, DRENAG = [m]\' vm La [=] VALOR SERVIÇOS: R$ 1.044.684,55 VALOR | DEDUÇÃO: R$ 0,00 DESC. INCOND: BASE DE R$ 0,00 CALCULO: R$ 1.044.684,55 ALÍQUOTA: 2% VALOR ISS: R$ 0,00 VALOR PIS: R$ 0,00 VALOR COFINS: R$ 0,00 VALOR IR: R$ 0,00 VALOR CSLL: R$ 0,00 OUTRAS R$ 0,00 eEd Era a e [=] Lt A =" DADOS VALOR INSS: R$ 11.491,53 VALOR ISS RETIDO: R$ 20.893,69 RETENÇÕES: DESC. COND: R$ 0,00 VALOR LÍQUIDO: R$ 1.012.299,33 COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS Exigivel REGIME TRIBUTAÇÃO Sociedade Limitada SIMPLES NACIONAL Não ISSQN RETIDO Sim LOCAL. PRESTAÇÃO SERVIÇO Macaé - RJ LOCAL INCIDÊNCIA Macaé - RJ Observação: - ISS RETIDO PELO TOMADOR DE SERVIÇOS CNPJ: 29.115.474/0001-60 Valor Aproximado dos Tributos Federais R$ 136154,26 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíq 0,00 IBPT) e Municipal de R$ 49906,36 (Alíq IBPT 4,93 IBPT) https://nfs-e.mage.rj.gov.br""",
            'official_name': 'Official Document Name',
            'min_score_similarity_analise': 0.45,
            'min_score_similarity_documento': 0.6
    }
}

tipo_documento_dict.update(novo_tipo_documento)

In [162]:
# XXX Salvando o dict do tipo de documento para arquivo json
save_tipo_doc_to_file(tipo_documento_dict, config_tipo_doc_path)

In [ ]:

# Exemplo de uso
loaded_dict = load_dict_from_file('config.json')


In [22]:
# XXX Criar um dicionário para matchers patterns - nfs-e
matcher_pattern_dict = {
        "numero_nota_pattern": [
            {"LOWER": "número"},
            {"LOWER": "da"},
            {"LOWER": "nota"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True}
        ],
        "competencia_pattern": [
            {"LOWER": "competência"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "?"},
            {"ORTH": {"REGEX": "^[A-Z][a-z]+/[0-9]{4}$"}}   
        ],
        "data_hora_emissao_pattern": [
            {"LOWER": "data"},
            {"LOWER": "e"},
            {"LOWER": "hora"},
            {"LOWER": "da"},
            {"LOWER": "emissão"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"SHAPE": "dd/dd/dddd"},
            {"IS_SPACE": True, "OP": "*"},
            {"SHAPE": "dd:dd:dd"}
        ],
        "codigo_verificacao_pattern": [
            {"LOWER": "código"},
            {"LOWER": "verificação"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_ASCII": True, "LENGTH": 9}
        ],
        "valor_total_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "total"},
            {"LOWER": "da", "OP": "?"},
            {"LOWER": "nota", "OP": "?"},
            {"TEXT": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_servicos_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "serviços"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_deducao_pattern": [
            {"LOWER": "dedução"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_incondR_pattern": [
            {"LOWER": "base"},
            {"LOWER": "de"},
            {"IS_SPACE": True},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"ORTH": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}    
        ],
        "valor_incond_patternP": [
            {"LOWER": "desc"},
            {"IS_PUNCT": True, "OP": "?"},
            {"LOWER": "incond"},
            {"ORTH": ":"},
            {"IS_SPACE": True},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"ORTH": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}    
        ],
        "valor_calculoR_pattern": [
            {"LOWER": "cálculo"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_calculoP_pattern": [
            {"LOWER": "base"},
            {"LOWER": "de"},
            {"LOWER": "cálculo"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_PUNCT": True, "OP": "?"},  # para lidar com possíveis quebras de linha
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ",", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_aliquota_pattern": [
            {"LOWER": "alíquota"},
            {"ORTH": ":"},
            {"SHAPE": "d,dd", "OP": "?"},
            {"ORTH": "%"}
        ],
        "valor_aliquota2_pattern": [
            {"LOWER": "alíquota"},
            {"ORTH": ":"},
            {"SHAPE": "d", "OP": "?"},
            {"ORTH": "%"}
        ],
        "valor_iss_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "iss"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_issretido_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "iss"},
            {"LOWER": "retido"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_desccond_pattern": [
            {"LOWER": "desc"},
            {"ORTH": "."},
            {"LOWER": "cond"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_pis_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "pis"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_cofins_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "cofins"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_ir_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "ir"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_inss_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "inss"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_csll_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "csll"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_outrasreten_pattern": [
            {"LOWER": "outras"},
            {"LOWER": "retenções"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "valor_liquido_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "líquido"},
            {"ORTH": ":"},
            {"SHAPE": "X$"},
            {"MORPH": "NumType=Card", "OP": "+"},
            {"LOWER": ".", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"}
        ],
        "exigibilidade_iss_pattern": [
            {"LOWER": "exigibilidade"},
            {"LOWER": "iss"},
            {"LOWER": {"IN": ["exigivel", "não exigivel"]}}
        ],
        "padrao_regime_tributacao": [
            {"LOWER": "regime"},
            {"LOWER": "tributação"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_ALPHA": True, "OP": "+"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_ALPHA": True, "OP": "*"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": "simples", "OP": "?"},
            {"IS_ALPHA": True, "OP": "*"}
        ],
        "simples_nacional_nao_pattern": [
            {"LOWER": "simples"},
            {"LOWER": "nacional"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": "não"}
        ],
        "simples_nacional_pattern": [
            {"LOWER": "simples"},
            {"LOWER": "nacional"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": "sim", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": "(", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True, "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": ",", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True, "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": "%", "OP": "?"},
            {"IS_SPACE": True, "OP": "*"},
            {"ORTH": ")", "OP": "?"}
        ],
        "issqn_retido_pattern": [
            {"LOWER": "issqn"},
            {"LOWER": "retido"},
            {"IS_SPACE": True, "OP": "*"},
            {"LOWER": {"IN": ["sim", "não"]}}
        ],
        "local_prestacao_servico_pattern": [
            {"LOWER": "local"},
            {"ORTH": "."},
            {"LOWER": "prestação"},
            {"LOWER": "serviço"},
            {"IS_SPACE": True, "OP": "+"},  # para lidar com múltiplos espaços
            {"IS_ALPHA": True, "OP": "+"},  # para a cidade
            {"ORTH": "-", "OP": "?"},
            {"IS_UPPER": True, "LENGTH": 2, "OP": "?"}  # para a sigla do estado
        ],
        "local_incidencia_pattern": [
            {"LOWER": "local"},
            {"IS_PUNCT": True, "OP": "?"},
            {"LOWER": "incidência"},
            {"IS_ALPHA": True, "OP": "+"},  # Nome da cidade
            {"ORTH": "-", "OP": "?"},  # Hífen opcional
            {"SHAPE": "XX", "OP": "?"}  # Sigla do estado
        ],
        "valor_aliquota_pattern": [
            {"LOWER": "valor"},
            {"LOWER": "iss"},
            {"ORTH": ":"},
            {"IS_SPACE": True, "OP": "*"},
            {"IS_DIGIT": True, "OP": "+"},
            {"ORTH": "", "OP": "?"},
            {"IS_DIGIT": True, "OP": "*"},
            {"ORTH": "%"}
        ]
}    

In [23]:
# Entity ruler patterns
colors = {
            "secretaria": "linear-gradient(90deg, #2ADB5E, #1FA346)", # Verde Degrade
            "tipo_documento": "linear-gradient(90deg, #09D6FF, #08A0D1)", #Azul medio degrade
            "nome_prefeitura": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", # Roxo claro para lilaz - degrade bem bacana
            "nome_section": "linear-gradient(90deg, #FFA9FB, #BF7FBC)", #  lilaz - Degrade
            "nome_section": "#FFEA7F", # Laranja claro
            "SAFRA": "#CCA10C", # Terracota
            "SAFRA": "#AB9BFC", # Roxo claro 
            "CNPJ": "#7AECEC", # Azul bem claro
            "NOME": "#EE8AF8" # Rosa medio
        }          

patternsPrefeitura = [
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mesquita"}], "id": "PM_MESQUITA"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "mage"}], "id": "PM_MAGE"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "sao"}, {"LOWER": "pedro"}, {"LOWER": "de"}, {"LOWER": "aldeia"}], "id": "PM_SPA"},
                        {"label": "nome_prefeitura", "pattern": [{"LOWER": "prefeitura"}, {"LOWER": "municipal"}, {"LOWER": "de"}, {"LOWER": "sao"}, {"LOWER": "pedro"}, {"LOWER": "da"}, {"LOWER": "aldeia"}], "id": "PM_SPA"}

                        ]


patternsSection = [     
                        {"label": "nome_section", "pattern": [{"LOWER": "número"}, {"LOWER": "da"}, {"LOWER": "nota"}, {"ORTH": ":"}], "id": "1. CABECALHO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "prestador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "2. PRESTADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "prestador"}], "id": "2. PRESTADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "tomador"}, {"LOWER": "de"}, {"LOWER": "serviços"}], "id": "3. TOMADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "tomador"}], "id": "3. TOMADOR DE SERVIÇO"},
                        {"label": "nome_section", "pattern": [{"LOWER": "discriminação"}, {"LOWER": "dos"}, {"LOWER": "serviços"}], "id": "4. DESCRIMINACAO DOS SERVIÇOS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "valor"}, {"LOWER": "total"}, {"LOWER": "da"}, {"LOWER": "nota"}], "id": "5. VALOR TOTAL"},
                        {"label": "nome_section", "pattern": [{"LOWER": "cnae"}], "id": "6. CNAE e Item da Lista de Serviços"},
                        {"label": "nome_section", "pattern": [{"LOWER": "valor"}, {"LOWER": "serviços"}], "id": "7. VALORES E IMPOSTOS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "dados"}, {"LOWER": "complementares"}], "id": "8. DADOS COMPLEMENTARES"},
                        {"label": "nome_section", "pattern": [{"LOWER": "outras"}, {"LOWER": "informações"}, {"IS_PUNCT": True}, {"LOWER": "criticas"}], "id": "9. OUTRAS INFORMAÇOES / CRITICAS"},
                        {"label": "nome_section", "pattern": [{"LOWER": "observação"}], "id": "10. OBSERVACOES"}

                        ]

patternsSecretarias = [{"label": "secretaria", "pattern": [{"LOWER": "secretaria"}, {"LOWER": "municipal"}, {"LOWER": "da"}, {"LOWER": "fazenda"},], "id": "SECRETARIA"}] 


patternsTipoDocumento = [
                        {"label": "tipo_documento", "pattern": [{"LOWER": "nota"}, {"LOWER": "fiscal"}, {"LOWER": "de"}, {"LOWER": "serviços"}, {"LOWER": "eletrônica"}, {"LOWER": "-"}, {"LOWER": "nfs-e"}], "id": "NFS-e"}
                        ]


patternsIdentificaEntidade = [
                            {"label": "CNPJ", "pattern": [{"ORTH": {"REGEX": "^\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}$"}}], "id": "cpf_cnpj_com_mascara"}
                            ]

patternsCnpj = [
    {
        "label": "CNPJ",
        "pattern": [
            {"ORTH": {"REGEX": "^\d{2}\.\d{3}\.\d{3}/$"}},
            {"ORTH": {"REGEX": "^\d{4}-\d{2}$"}}
        ]
    }
]


patterns = patternsPrefeitura + patternsSection + patternsSecretarias + patternsTipoDocumento + patternsIdentificaEntidade + patternsCnpj

In [24]:
# XXX Processo para salvar matcher pattern para json
nome_matches_pattern_json = tipo_documento + "_matcher_pattern.json"
path_matches_pattern_json = os.path.join(tipo_documento_patterns_path, nome_matches_pattern_json)
with open(path_matches_pattern_json, "w") as f:
    json.dump(matcher_pattern_dict, f)

In [25]:
# XXX Processo para salvar entity ruler pattern para json
nome_entityruler_pattern_json = tipo_documento + "_entity_ruler_pattern.json"
entityruler_file_path = os.path.join(tipo_documento_patterns_path, nome_entityruler_pattern_json)

write_patterns_to_file(patterns=patterns, colors=colors, filename=entityruler_file_path)

In [11]:
# XXX Rotina para carregar e atribuir ao matcher os patterns do disco

matcher = Matcher(nlp.vocab)
# XXX Carregar matcher patterns do disco
with open("config/tipo_documentos/patterns/nfs_e_matcher_pattern.json", "r") as f:
    loaded_patterns = json.load(f)
    
# Adicionar ao Matcher
for label, pattern in loaded_patterns.items():
    matcher.add(label, [pattern])    

In [ ]:
# XXX Carregar Entity Ruler patterns do disco
patterns, colors = load_patterns_and_colors("config/tipo_documentos/patterns/nfs_e_entity_ruler_pattern.json")


# XXX Aplicar a funçao show_ent_new para exibir o resultado
doc, tokens, ents = show_ent_new(texto_PDF, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

In [ ]:
# XXX Analisar os Matcher patterns do documento

matches = matcher(doc)

# Exibir os resultados
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Obter a string de identificação
    span = doc[start:end]  # Obter o trecho correspondente
    print(f"{string_id}: {span.text}")

In [ ]:
doc1 = tipo_documento_dict['sample_content']
doc1

In [107]:
# 0.B Extracao de texto de todo o documento - RASTER PDF	
def extrai_texto_R_PDF(file_path, output_file, debug):    
    
    input_file = file_path

    # 1. XXX Executar o comando OCRmyPDF
    run_ocrmypdf(input_file, output_file)

    # 2. XXX Executar o comando OCRmyPDF    
    !pdftotext processamentos/temp/documento.pdf processamentos/temp/txt/documento.txt

    # 3. XXX Ler o arquivo TXT
    with open('processamentos/temp/txt/output.txt', 'r', encoding='utf-8') as arquivo:
        texto_OCR_R = arquivo.read()
        
    texto_PDF_Raster = re.sub('\s+', ' ', texto_OCR_R).strip()
    
    if debug:
        print(f'\nFUNC extrai_texto_PDF_P: doc.:{original_file_name} | diretorio: {map_directory}  texto_PDF_Raster: \n\n{texto_PDF_Raster}\n\n')

    return texto_PDF_Raster

In [ ]:
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_23/MESQUITA_PDF_31282023_2258/159871/2023 -4.pdf"
document_unique_id = utl.generate_unique_id()
nome_arquivo = os.path.basename(file_path)

OCRmyPDF completed successfully. Output saved to processamentos/temp/pdf/6bca0884-16ee-4e13-9d23-c20a6074924c.pdf.


"29/07/23, 15:50 Nota Fiscal de Serviços Eletrônica (NFSe) Número da Nota: PREFEITURA MUNICIPAL DE MAGE ia: Competência: SECRETARIA MUNICIPAL DA FAZENDA Julho/2023 NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e Data e Hora da Emissão: 25/07/2023 15:40:00 Código Verificação: B16C73D79 PRESTADOR DE SERVIÇOS CPF/CNPJ: 33.462.862/0001-95 Inscrição Municipal: 4337 Telefone: Inscrição Estadual: nr: a Nome/Razão Social: SÃO MARCOS TERRAPLENAGEM E CONST RUÇÃO LTDA. Nome de Fantasia: Endereço: RUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ E-mail: TOMADOR DE SERVIÇOS CPF/CNPJ: 09.542.728/0001-10 INSC:MUNICIPAL: RG: | Inscrição Estadual: Telefone: Nome/Razão Social: CONSORCIO ZADAR - ENGETECNICA Endereço: RUA SENADOR DANTAS, 75 Nº 75 SALA 1903 BAIRRO: CENTRO CIDADE: RIO DE JANEIRO - RJ CEP: 20031914 E-mail: NÃO INFORMADO DISCRIMINAÇÃO DOS SERVIÇOS Serviço de Pavimentação Asfáltica. Linha Vermelha, conf Med 13/23. M.Obra (10%)=R$ 3.596,97 Retenção p/ Prev. Social = R$ 395,67 BANCO : ITAÚ AGÊNCIA: 07

In [213]:
file_path = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_raster/Mage/Techmuniz 0032 Sys Manager.pdf"
document_unique_id = utl.generate_unique_id()
debug = False

In [ ]:
texto_PDF_Raster = extracao_texto_PDF_Raster(document_unique_id, file_path, debug)
texto_PDF_Raster

In [ ]:
doc, tokens, ents = show_ent_new(texto_PDF_Raster, patterns=patterns)

displacy.render(doc, style="ent", options={"colors": colors})

In [ ]:
texto_padrao = tipo_documento_dict.get('nfs_e', {}).get('sample_content', 'valor_padrao')
texto_padrao

In [218]:
doc1, tokens, ents = show_ent_new(texto_padrao, patterns=patterns)

In [146]:
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_23/MESQUITA_PDF_31282023_2258/159871/2023 -4.pdf"
document_unique_id = utl.generate_unique_id()
nome_arquivo = os.path.basename(file_path)

In [164]:
score_min_documento = tipo_documento_dict.get('nfs_e', {}).get('min_score_similarity_documento', 'valor_padrao')
score_min_documento

0.7

In [165]:
score_min_analise = tipo_documento_dict.get('nfs_e', {}).get('min_score_similarity_analise', 'valor_padrao')
score_min_analise

0.6

In [222]:
doc1 = texto_padrao
doc2 = texto_PDF_Raster

Score de Similaridade: 0.7324063189820084

Quase certeza que e NFS-e pois seu score e: 0.7324063189820084 e o score minimo documento e de: 0.7 - acao_sugerida: PREPROCESS_EXTRACT



In [ ]:
# Unica funcao que devera ser movida para junto das demais funcoes de dicionario
def define_rotulo_acao(nome_arquivo):
    
    for palavra_chave, rotulo in mapeamento_palavras_chave.items():
        if palavra_chave.lower() in nome_arquivo.lower():
            break
    else:
        rotulo = 'prov_nota_fiscal' #"sem_rotulo"
        palavra_chave = 'default'
        acao_sugerida = sugestoes_acao.get(rotulo, 'None')
        return palavra_chave, rotulo, acao_sugerida
        # palavra_chave = 'None' #"sem_palavra_chave"
        # acao_sugerida = 'None' #"sem_acao_sugerida"
        
        return palavra_chave, rotulo, acao_sugerida
        #print(f'nome_arquivo: {nome_arquivo} | rotulo: {rotulo}')
    if rotulo != 'None': #"sem_rotulo"
        acao_sugerida = sugestoes_acao.get(rotulo, 'None') # "Ação não definida"
        return palavra_chave, rotulo, acao_sugerida
    
    
    
    # Dicionário para mapear palavras-chave a rótulos
mapeamento_palavras_chave = {
    "relatorio": "prov_relatorio",
    "listagem": "prov_listagem",
    "NF": "prov_nota_fiscal",
    "nf": "prov_nota_fiscal",
    "relatorio": "prov_listagem",
    "sintetico": "prov_listagem",
    "livro": "prov_livro_registro",
    "sintético": "prov_listagem",
    "nota": "prov_nota_fiscal",
    "zip": "doc_zip",
    "rar": "doc_rar",
    "valores": "prov_dinheiro",
}

# Dicionário mapeando rótulos a ações sugeridas
sugestoes_acao = {
    "prov_relatorio": "NO_PROCESS",
    "prov_listagem": "NO_PROCESS",
    "prov_nota_fiscal": "PROCESS",
    "sem_rotulo": "MANUAL_REV",
    "prov_livro_registro": "NO_PROCESS",
    "doc_nao_pdf": "verificar",
    "nao_pdf": "NO_PROCESS",
    "doc_zip": "NO_PROCESS",
    "pdf_mul_paginas": "SPLIT",
}



# 2.Testando
nome_arquivo = 'batatinha_quando_nasce.pdf' # 'pre-processamento'
#palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo, debug)
#print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    

## <mark> <b> 3.0</b>  Mapeamento e Analise do pipeline </mark>

In [ ]:
# 1. XXX apagar arquivos zone
utl.apagar_zone(documentos_extracao_path)

In [ ]:
# SOMENTE PARA TESTE DE FUNCAO
#fake_parent_document_unique_id = 'f976c128-1f41-4551-bffd-fac687c1c8b2'

# Busca proximo Batch caso nao esteja rodando email
batch_name = utl.busca_proximo_batch(conf_export_plan_path)
batch_name

In [ ]:
# 0.B XXX Funçao de criacao do PDF Pesquisavel
def run_ocrmypdf(input_file, output_file):
    command = [
        'ocrmypdf',
        '--language', 'por',
        '--deskew',
        '--force-ocr',
        input_file,
        output_file
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode == 0:
        print(f"OCRmyPDF completed successfully. Output saved to {output_file}.")
    else:
        print(f"OCRmyPDF failed with error: {result.stderr.decode('utf-8')}")

In [228]:
# 1.A XXX Extracao de texto de todo o documento - PDF PESQUISAVEL	
def extracao_texto_PDF_P(i, document_unique_id, nome_arquivo, file_path, debug):    
    
   # Carregar o arquivo PDF
    pdf_document = fitz.open(file_path)

    # Página do PDF  ATENCAO  (UNICA PAGINA)
    page_number = 0  # Defina o número da página que deseja analisar
    page = pdf_document[page_number]

    # Extrair texto dentro do retângulo
    text_P = page.get_text("text")
    
    pdf_document.close()
    
    texto_PDF = re.sub('\s+', ' ', text_P).strip()  
    if debug:
        print(f'\nFUNC analise_texto_PDF: doc.:{nome_arquivo}   texto_PDF_P: \n\n{texto_PDF}\n\n')

    return texto_PDF_P

In [227]:
# 1.B XXX Extracao de texto de todo o documento - RASTER PDF
def extracao_texto_Raster_P(i, document_unique_id, nome_arquivo, file_path, debug):    
    
    output_file = None
    txt_document_file = None
    
    input_file = file_path
    
    #output_file = "/home/dani-boy/extractNF/processamentos/temp/documento.pdf"
    output_file = os.path.join(raster_process_pdf_path, document_unique_id + '.pdf')

    # 1. XXX Executar o comando OCRmyPDF
    run_ocrmypdf(input_file, output_file)

    txt_document_file = os.path.join(raster_process_txt_path, document_unique_id + '.txt')

    txt_dir = os.path.dirname(txt_document_file)
    if not os.path.exists(txt_dir):
        os.makedirs(txt_dir)

    # Execute o comando
    subprocess.run(["pdftotext", output_file, txt_document_file])

    # 3. XXX Ler o arquivo TXT
    with open(txt_document_file, 'r', encoding='utf-8') as arquivo:
        texto_OCR_R = arquivo.read()
    texto_PDF_Raster = re.sub('\s+', ' ', texto_OCR_R).strip()
    
    os.remove(output_file)
    os.remove(txt_document_file)
    
    return texto_Raster_P

In [ ]:
# 2. XXX Analise de silimaridade doc1 x doc2
def analisa_similiaridade_doc(doc1, doc2):
    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([doc1, doc2])

    # Calculando Similaridade de Cosseno
    similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])

    print(f"Score de Similaridade: {similarity_scores[0][0]}")
    
    return similarity_scores[0][0]

In [ ]:
score = analisa_similiaridade_doc(doc1, doc2)

if score > score_min_documento:
    acao_sugerida = 'PREPROCESS_EXTRACT'
    print(f'\nQuase certeza que e NFS-e pois seu score e: {score} e o score minimo documento e de: {score_min_documento} - acao_sugerida: {acao_sugerida}\n')
elif score > score_min_analise: 
    acao_sugerida = 'PREPROCESS_ANALISE'
    print(f'\nPode ser que se trate de um documento NFS-e pois seu score e: {score} e o score minimo e de analise e: {score_min_analise} - acao_sugerida: {acao_sugerida}\n')   
elif score < score_min_analise:
    acao_sugerida = 'NO_PROCESS'
    print(f'\nE pouco provavel que seja NFSe pois seu score e de: {score} para um score minimo de: {score_min_analise} - acao_sugerida: {acao_sugerida}\n')    

In [ ]:
# 1.XXX  Acao 1 - Ler todo o pipeline de documentos recebidos - ESSA E A UNICA FUNCAO QUE ITERA NO DIRETORIO
def scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status, debug):
    
    doc_info = {}
    resumo = {}
    raw_document = []
    
    output_dir = os.path.join(documentos_extracao_path, batch_name)
    i = 1
    for root, dirs, files in os.walk(output_dir):
        folder_name = os.path.basename(root)
        #print(folder_name)
        for file in files:
            pdf_pesquisavel_map = None
            document_unique_id = utl.generate_unique_id()
            nome_arquivo = file
            
            palavra_chave, rotulo, acao_sugerida = define_rotulo_acao(nome_arquivo)
            acao_executada = "Analise"
            informations = ' '    
            file_name = file.lower()    
            file_path = os.path.join(root, file)
            if file.lower().endswith('.zip') or file.lower().endswith('.rar') or file.lower().endswith('.7z'):
                compressed_file_path = file_path
                compressed_file_name = os.path.basename(compressed_file_path)
                compressed_file_hash = utl.generate_file_hash(compressed_file_path)
                compressed_file_unique_id = utl.generate_unique_id()
                parent_document_unique_id = compressed_file_unique_id
                level = 2
            new_path_name = os.path.join(output_dir, file)
            acao_sugerida = 'ARCHIEVE_EXTRACTION'
            
            if file.lower().endswith('.pdf'):
                doc_one_page, nro_pgs = tpdf.analisa_nro_pages(file_path)
                
                one_page = doc_one_page
                num_page = nro_pgs
                if num_page > 1:
                    acao_sugerida = 'SPLIT_PAGES'
                else:
                    texto_PDF = analise_texto_PDF(i, document_unique_id, nome_arquivo, file_path, debug)  
                    level = 3
                    if texto_PDF:
                        pdf_pesquisavel_map = True
                    else:
                        pdf_pesquisavel_map = False  
                
                
                
                
                

            
            else:
                one_page = False
                    
            # if doc_one_page:
            #     one_page = doc_one_page
            # else:
            #     one_page = False    
            #             rotulo = 'pdf_mul_paginas'
            diretorio = os.path.basename(file_path)
            if folder_name == batch_name:
                folder_name = "root_dir"
                
            #print(f'nome_arquivo: {nome_arquivo:>55} | palavra_chave: {palavra_chave:>20} | rotulo: {rotulo:20} | acao_sugerida: {acao_sugerida:30}')    
            
            new_row = {
                "seq": i,
                "date_time": cron.timenow_pt_BR(),
                "batch": batch_name,
                "fase_processo": fase,
                "nome_atividade": atividade,
                "status_documento": status,
                "acao_executada": acao_executada,
                "original_file_name": file,
                "directory": folder_name,
                "one_page": one_page,
                "pages": num_page,
                "palavra_chave": palavra_chave,
                "document_tag": rotulo,
                "action_item": acao_sugerida,
                "level": level,
                "pdf_pesquisavel": pdf_pesquisavel_map,
                "document_unique_id": document_unique_id,
                "parent_document_unique_id": parent_document_unique_id,
                "file_hash": utl.generate_file_hash(file_path),
                "file_path": file_path,
                "informations": informations,
            }
            raw_document.append(new_row)

            
            # print(f'seq: {i} | file: {file}'
            i += 1
    df_trans_pipe = pd.DataFrame(raw_document)
      
                
    return df_trans_pipe, raw_document

In [ ]:
# 1. XXX Crio o DF df_scan_pipe
fase = 'analise' # 'pre-processamento'
atividade = 'scan_analise'
status = 'root_analise'

documentos = []
debug = False

df_root_pipe, documentos = scan_pipeline_documentos(documentos_extracao_path, batch_name, fase, atividade, status, debug)

In [ ]:
# 0. Acertando o Index
df_root_pipe.set_index('document_unique_id', inplace=True)
df_root_pipe

### exportando o map_analise_path

In [ ]:
# 1. XXX Definiçao do path para salvar o arquivo
file_path_root_pipe = os.path.join(map_analise_path, df_root_pipe_file + batch_name + ".xlsx")

# 2. XXX Salvando o arquivo de df: df_root_pipe
df_root_pipe.to_excel(file_path_root_pipe, index=True)

## Pesquisas e outros

### Atualizando o valor da coluna status_documento

In [ ]:
def update_status(row):
    if row['seq'] in sequencia:
        row['status_documento'] = 'PREPROCESS_EXTRACT'
    return row

In [ ]:
sequencia = [2, 8, 6]

In [ ]:
df_root_pipe = df_root_pipe.apply(update_status, axis=1)
df_root_pipe

In [ ]:
# Podemos chegar de forma mais rapida
subset_df.loc[subset_df['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
subset_df_analise_pipe = df_analise_pipe.query('seq == 59')

In [ ]:
# XXX Salvando o DF (IMPORTANTE)
df_analise_pipe.to_excel("df_mapeamento_e_analise2.xlsx", index=True)

In [ ]:
# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)

In [ ]:
# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')

In [ ]:
# 7. XXX Retiro o indice do DF - Resetando o índice e mantendo o índice original como uma nova coluna
df_analise_pipe.reset_index(inplace=True)


# XXX Criando uma nova coluna no DF
df_conferencia.insert(loc=50, column='original_file_name', value=df_conferencia['file_path'].apply(lambda x: os.path.basename(x)))

df['coluna_cnae'] = df['coluna_cnae'].apply(lambda x: x.strip() if isinstance(x, str) else x)


# 4. XXX Usando loc para Filtrar Baseado em one_page == False
df_pages_2_split = df_root_pipe[df_root_pipe['one_page'] == False]



# 3. Usando query para Filtrar Baseado em Condições Complexas
df_root_pipe.query('one_page == False & palavra_chave == "sem_palavra_chave"')


# XXX Criando uma nova coluna no DF
df_analise_pipe.insert(loc=17, column='s_act', value=None)


# 4. Filtrando Linhas Baseadas em Valores em uma Lista

valores = [11, 16, 30, 41]
subset_df = df_scan_pipe[df_scan_pipe['seq'].isin(valores)]

# 2. Usando iloc para Filtrar um Número Específico de Linhas
subset_df = df_scan_pipe.iloc[:10]




registro_específico = df_scan_pipe.loc['e1f4b1af-30f3-45d2-85a7-1bb895bd5325']



df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: row['coluna1'] * 2 if row['coluna2'] > 0 else row['coluna1'], axis=1)


# Atualizei o DF baseado em condiçao de outra coluna
df_analise_pipe['pdf_pesquisavel'] = df_analise_pipe.apply(lambda row: False if row['status_documento'] == "Template_encontrado" else row['pdf_pesquisavel'], axis=1)


# Podemos chegar de forma mais rapida
df_analise_pipe.loc[df_analise_pipe['status_documento'] == "Template_encontrado", 'pdf_pesquisavel'] = False


# 8. XXX Concatenando os DataFrames
df_analise_pipe = pd.concat([df_analise_pipe, df_docs_splitados], ignore_index=True)

# 9. XXX Volto novamente o indice do DF
df_analise_pipe.set_index('document_unique_id', inplace=True)

In [ ]:
output_file = None
txt_document_file = None
input_file = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_raster/Mage/nf_688___consor_zadar__engetecnica_enza_1_anotado.pdf"
document_unique_id = utl.generate_unique_id()
#output_file = "/home/dani-boy/extractNF/processamentos/temp/documento.pdf"
output_file = os.path.join(raster_process_pdf_path, document_unique_id + '.pdf')

# 1. XXX Executar o comando OCRmyPDF
run_ocrmypdf(input_file, output_file)

txt_document_file = os.path.join(raster_process_txt_path, document_unique_id + '.txt')

txt_dir = os.path.dirname(txt_document_file)
if not os.path.exists(txt_dir):
    os.makedirs(txt_dir)

# Execute o comando
subprocess.run(["pdftotext", output_file, txt_document_file])

# 3. XXX Ler o arquivo TXT
with open(txt_document_file, 'r', encoding='utf-8') as arquivo:
    texto_OCR_R = arquivo.read()
texto_PDF_Raster = re.sub('\s+', ' ', texto_OCR_R).strip()
texto_PDF_Raster


